In [9]:
import sys
from pathlib import Path
ROOT_PATH = str('../sfs')
sys.path.append(ROOT_PATH)
from shapwise_feature_selector import *
np.random.seed(0)
%matplotlib inline

In [6]:
import sys
from pathlib import Path
cwd = Path.cwd()
ROOT_PATH = str('../../op-data-science-prod')

sys.path.append(ROOT_PATH)

from simpml.tabular.all import *
np.random.seed(0)
%matplotlib inline

from feature_engine.discretisation import EqualWidthDiscretiser

class NumericToStringTransformer(BaseEstimator, TransformerMixin):
    """
    A transformer that converts numeric values into their string representation.
    """
    
    def fit(self, X, y=None):
        """
        Fit does nothing in this case as no fitting is necessary.
        :param X: Input data, array-like, shape (n_samples, n_features)
        :param y: Not used, present for API consistency by convention.
        :return: self
        """
        # No fitting process required as we're just converting types
        return self
    
    def transform(self, X):
        """
        Transform numeric values in the dataset to strings.
        :param X: Input data, array-like, shape (n_samples, n_features)
        :return: Transformed data with numeric values converted to strings.
        """
        # Apply the transformation column-wise
        if isinstance(X, pd.DataFrame):
            for column in X.columns:
                X[column] = X[column].astype(str)
        else:
            X = X.astype(str)
        return X

In [7]:
data_manager = SupervisedTabularDataManager(data = DataSet.load_titanic_dataset(),
                                            target = 'Survived',
                                            prediction_type = PredictionType.BinaryClassification,
                                            splitter = 'RCA')
data_manager.build_pipeline(drop_cols = ['PassengerId'])
data_manager.pipeline.sklearn_pipeline.steps.insert(9, ('EqualFrequencyDiscretiser', EqualWidthDiscretiser(bins = 3)))
data_manager.pipeline.sklearn_pipeline.steps.insert(10, ('NumericToStringTransformer', NumericToStringTransformer()))
exp_mang = ExperimentManager(data_manager, optimize_metric = MetricName.AUC)
exp_mang.remove_models(['Logistic Regression','Support Vector Classifier','AdaBoost Classifier','LightGBM'])
exp_mang.run_experiment(metrics_kwargs = {'pos_label': 1})

,Experiment ID,Experiment Description,Model,Model Description,Data Version,Data Description,Model Params,Metric Params,Accuracy,AUC,Recall,Precision,Balanced Accuracy,F1,Run Time
0,20240218191211_97b7,,Baseline Classification,Default settings,55f87c5f,,"{'experiment_manager': Prediction Type: PredictionType.BinaryClassification, Metric: Metric: AUC. Description: , Random State: 0}",{'pos_label': 1},0.497194,0.466302,0.333333,0.341317,0.466302,0.337278,0:00:00
1,20240218191211_97b7,,Decision Tree,Default settings,55f87c5f,,"{'experiment_manager': Prediction Type: PredictionType.BinaryClassification, Metric: Metric: AUC. Description: , Random State: 0}",{'pos_label': 1},0.875421,0.848193,0.730994,0.929368,0.848193,0.818331,0:00:00
2,20240218191211_97b7,,Random Forest,Default settings,55f87c5f,,"{'experiment_manager': Prediction Type: PredictionType.BinaryClassification, Metric: Metric: AUC. Description: , Random State: 0}",{'pos_label': 1},0.875421,0.850398,0.742690,0.916968,0.850398,0.820679,0:00:00
3,20240218191211_97b7,,Gradient Boosting,Default settings,55f87c5f,,"{'experiment_manager': Prediction Type: PredictionType.BinaryClassification, Metric: Metric: AUC. Description: , Random State: 0}",{'pos_label': 1},0.868687,0.841626,0.725146,0.915129,0.841626,0.809135,0:00:00
4,20240218191211_97b7,,XGBoost,Default settings,55f87c5f,,"{'experiment_manager': Prediction Type: PredictionType.BinaryClassification, Metric: Metric: AUC. Description: , Random State: 0}",{'pos_label': 1},0.830527,0.795777,0.646199,0.880478,0.795777,0.745363,0:00:00


In [14]:
X_train, y_train = data_manager.get_training_data()
X_test, y_test = data_manager.get_validation_data()

In [15]:
sfs_model = SHAPwiseFeatureSelector(exp_mang.best_model.model, accuracy_score, number_top_fi = 30)

In [16]:
sfs_model.fit(X_train, y_train, X_test, y_test)

Bad noisy feature found: Embarked_Q
old score: 0.8754 new score: 0.8754


SHAPwiseFeatureSelector(base_estimator=RandomForestClassifier(n_jobs=-1,
                                                              random_state=42),
                        metric=<function accuracy_score at 0x7f545140fee0>,
                        number_top_fi=30)